<a href="https://colab.research.google.com/github/jrojasquiroz/peru_sno/blob/main/prepdata_peru_sno.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Por separado, tenía el archivo del polígono urbano de las principales ciudades del Perú, las manzanas de todo el país con su respectiva población (ambos datos identificados por INEI con base en el Censo de 2017), y las regiones naturales.

Lo que hago en este cuaderno es preparar los datos para luego analizarlos. Esta preparación no consiste en más que unir todos esos datos en un único archivo que yo exporto a la carpeta `dataviz/orientacion/pu-principalesciudades-WGS84.gpkg` de mi Drive, tú debes seleccionar una dentro de tu directorio.


In [1]:
pip install geopandas

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import geopandas as gpd

In [4]:
# Cargar los polígonos urbanos de las ciudades
gdf_cities = gpd.read_file('/content/drive/MyDrive/Datos/vectores/base/Manzanas Perú (INEI)/pu-principalesciudades-og_WGS84.gpkg')

# Cargar las manzanas con la población
gdf_manzanas = gpd.read_file('/content/drive/MyDrive/Datos/vectores/base/Manzanas Perú (Geo GPS Perú)/peru-mzsurb_WGS84.gpkg')

# Cargar las regiones naturales
gdf_regiones = gpd.read_file('/content/drive/MyDrive/Datos/vectores/base/peru-regiones_nat-WGS84.gpkg')

In [5]:
# ------------------- INTERSECCIÓN CON MANZANAS Y SUMA DE POBLACIÓN -------------------

# Realizar la intersección espacial entre las ciudades y las manzanas
# Esto devuelve solo las manzanas que caen dentro de los límites de cada ciudad
intersected_manzanas = gpd.overlay(gdf_manzanas, gdf_cities, how='intersection')

# Sumar la población de las manzanas dentro de cada ciudad
# La columna de población en el archivo de manzanas es 'T_TOTAL', y agrupamos por el identificador de la ciudad
pop_by_city = intersected_manzanas.groupby('CIUDAD')['T_TOTAL'].sum().reset_index()

# Unir la población a los datos de las ciudades
# Nos aseguramos de que el nombre de la ciudad sea la clave para la unión
gdf_cities = gdf_cities.merge(pop_by_city, on='CIUDAD', how='left')

# Renombrar la columna de población como 'POB17'
gdf_cities.rename(columns={'T_TOTAL': 'POB17'}, inplace=True)

# ------------------- INTERSECCIÓN CON REGIONES NATURALES -------------------

# Realizar la intersección espacial entre las ciudades y las regiones naturales
intersected_regiones = gpd.overlay(gdf_cities, gdf_regiones, how='intersection')

# Mantener solo las columnas necesarias, es decir, agregar la columna 'Nm_RegNat' de las regiones naturales
# Asegurarse de que no haya duplicados después de la intersección
intersected_regiones = intersected_regiones[['CIUDAD', 'Nm_RegNat']].drop_duplicates()

# Unir la información de la región natural a los datos de las ciudades
gdf_cities = gdf_cities.merge(intersected_regiones, on='CIUDAD', how='left')

# Renombrar la columna de la región natural como 'REGNAT'
gdf_cities.rename(columns={'Nm_RegNat': 'REGNAT'}, inplace=True)

# Mostrar el resultado
print(gdf_cities[['CIUDAD', 'POB17', 'REGNAT']])

                CIUDAD      POB17  REGNAT
0              ABANCAY    71781.0  Sierra
1          ANDAHUAYLAS    57823.0  Sierra
2             AREQUIPA  1006502.0  Sierra
3             AYACUCHO   217551.0  Sierra
4              AYAVIRI    21430.0  Sierra
..                 ...        ...     ...
87                VIRU    20297.0   Costa
88               YAURI    28401.0  Sierra
89          YURIMAGUAS    62384.0   Selva
90           ZARUMILLA    36525.0   Costa
91  LIMA METROPOLITANA  9141029.0   Costa

[92 rows x 3 columns]


In [6]:
gdf_cities.to_file('/content/drive/MyDrive/Documentos personales/1-Investigaciones y análisis/dataviz/orientacion/pu-principalesciudades-WGS84.gpkg')